In [28]:
import json
from copy import deepcopy

In [3]:
with open("data/text/whisper.json", "r") as f:
    whisper = json.load(f)

In [27]:
with open("data/text/diarization.json", "r") as f:
    diarization = json.load(f)

In [56]:
speaker_dict = {
    "SPEAKER_00": "Sorter",
    "SPEAKER_01": "Blacksheep",
    "SPEAKER_02": "Gergő",
    "SPEAKER_03": "FreddyD",
}

In [57]:
diary = deepcopy(diarization)

transcript = []
speaker = None
for x in whisper:
    new_start = x["start"]
    new_end = x["end"]
    new_text = x["text"]
    new_speaker = None
    for i, y in enumerate(diary):
        if new_start <= y["stop"] and new_end >= y["start"]:
            new_speaker = speaker_dict[y["speaker"]]
            diary = diary[i:]
            break
    
    # intro
    if new_speaker is None:
        continue
    
    # first speaker
    if speaker is None:
        speaker_start = new_start
        speaker_end = new_end
        speaker = new_speaker
        speaker_text = new_text
        continue
    
    # if old speaker is same as current speaker
    if new_speaker == speaker:
        speaker_text += " " + new_text
        speaker_end = new_end
    # else: speaker changed, add old speaker to transcript
    else:
        transcript.append({"start": speaker_start, "end": speaker_end, "speaker": speaker, "text": speaker_text})
        speaker_start = new_start
        speaker_end = new_end
        speaker = new_speaker
        speaker_text = new_text

In [60]:
with open("analysis/transcript.json", "w") as f:
    json.dump(transcript, f, indent=4)